In [1]:
import os 
import cv2 
import numpy as np 
from glob import glob 
from scipy.io import loadmat
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers

C:\Users\oceanlightai\anaconda3\envs\gpu\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\oceanlightai\anaconda3\envs\gpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\oceanlightai\anaconda3\envs\gpu\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
IMAGE_SIZE = 512
BATCH_SIZE = 4
NUM_CLASSES = 6
DATA_DIR = "C:/Users/oceanlightai/Desktop/datasets/pet_skin/train"
NUM_TRAIN_IMAGES = 1000
NUM_VAL_IMAGES = 50

In [3]:
train_images = sorted(glob(os.path.join(DATA_DIR, 'train_Image2/*')))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(DATA_DIR, 'train_mask2/*')))[:NUM_TRAIN_IMAGES]

val_images = sorted(glob(os.path.join(DATA_DIR, 'train_Image2/*')))[NUM_TRAIN_IMAGES:NUM_VAL_IMAGES+NUM_TRAIN_IMAGES]
val_masks = sorted(glob(os.path.join(DATA_DIR, 'train_mask2/*')))[NUM_TRAIN_IMAGES:NUM_VAL_IMAGES+NUM_TRAIN_IMAGES]

In [4]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_jpeg(image, channels=1)
        image.set_shape([None,None,1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        
    else:
        image = tf.image.decode_jpeg(image, channels=3)
        image.set_shape([None,None,3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 127.5 - 1
        
    return image

In [5]:
def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

In [6]:
def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [7]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("train dataset:" , train_dataset)
print("valid_dataset:", val_dataset)

train dataset: <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.float32)>
valid_dataset: <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.float32)>


In [8]:
def convolution_block(block_input, num_filters=256, kernel_size=3, 
                     dilation_rate=1, padding='same', use_bias=False):
    x = layers.Conv2D(num_filters, kernel_size=kernel_size, 
                      dilation_rate=dilation_rate, padding='same', 
                      use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)

In [9]:
def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(size=(dims[-3]// x.shape[1], dims[-2]// x.shape[2]),
                                         interpolation='bilinear')(x)
    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
    
    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    
    output = convolution_block(x, kernel_size=1)
    return output

In [10]:
def DeeplabV3(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    
    x = resnet50.get_layer('conv4_block6_2_relu').output
    x = DilatedSpatialPyramidPooling(x)
    
    input_a = layers.UpSampling2D(size=(image_size // 4 // x.shape[1],
                                       image_size // 4 // x.shape[2]),
                                       interpolation='bilinear')(x)
    input_b = resnet50.get_layer('conv2_block3_2_relu').output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)
    
    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(size=(image_size // x.shape[1],
                                 image_size // x.shape[2]),
                           interpolation='bilinear')(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1,1), padding='same')(x)
    
    return keras.Model(inputs=model_input, outputs=model_output)

model = DeeplabV3(image_size=IMAGE_SIZE, num_classes = NUM_CLASSES)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), 
              loss=loss,
              metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=val_dataset, epochs=30)

Epoch 1/30
250/250 [==============================] - 47s 152ms/step - loss: nan - accuracy: 0.9935 - val_loss: nan - val_accuracy: 0.9873
Epoch 2/30
250/250 [==============================] - 38s 152ms/step - loss: nan - accuracy: 0.9968 - val_loss: nan - val_accuracy: 0.9873
Epoch 3/30
250/250 [==============================] - 38s 151ms/step - loss: nan - accuracy: 0.9968 - val_loss: nan - val_accuracy: 0.9873
Epoch 4/30
 50/250 [=====>........................] - ETA: 29s - loss: nan - accuracy: 0.9960

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-6), 
              loss=loss,
              metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=val_dataset, epochs=30)